In [ ]:
from snowflake.snowpark import Session
import logging
import time
import os

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("SNOWFLAKE_TASKS")

# Snowflake Connection
connection_params = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA")
}

#session = Session.builder.configs(connection_params).create()

def execute_stored_procedure(procedure_name):
    """Executes a stored procedure using the provided session."""
    try:
        logger.info(f"Executing stored procedure: {procedure_name}")
        session.sql(f"CALL {procedure_name}();").collect()
        logger.info(f"Stored Procedure `{procedure_name}` executed successfully.")
    except Exception as e:
        logger.error(f"Error executing {procedure_name}: {str(e)}")

def check_task_status():
    """Checks the latest execution status of tasks in Snowflake."""
    try:
        logger.info("Fetching latest task execution status...")
        df = session.sql("""
            SELECT NAME, STATE, COMPLETED_TIME 
            FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY()) 
            ORDER BY COMPLETED_TIME DESC LIMIT 5
        """).collect()
        for row in df:
            logger.info(f"Task: {row['NAME']} | Status: {row['STATE']} | Completed: {row['COMPLETED_TIME']}")
    except Exception as e:
        logger.error(f"Error fetching task status: {str(e)}")

# Step 1: Execute Stored Procedures Using the Provided Session
#execute_stored_procedure("PROCESS_TOM_PROCESSED_STREAM()")
execute_stored_procedure("PROCESS_TOM_PROCESSED_TASK")
time.sleep(5)  # Wait for the first task to complete before triggering the second
execute_stored_procedure("PROCESS_HARMONIZED_TOM_TASK")

# Step 2: Monitor the Task Execution
check_task_status()

print("Stored Procedure Triggered & Execution Status Checked!")
